##Part 2
#Automatic detection of fraudulous signals using Machine Learning and Deep Learning 

## MVA Project for the course "Deep Learning and Signal detection: Introduction and industrial applications" 

# Environment 


In [ ]:
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
from scipy.stats.stats import pearsonr  
from sklearn.neighbors import KDTree
from sklearn.metrics import accuracy_score
import json
import pickle
from torch.utils.data import TensorDataset, DataLoader

from torch.optim.lr_scheduler import StepLR
import torch

#data prep

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
path_to_data = '/content/drive/MyDrive/Projet_II/'

In [ ]:
#loading the labels of the data

with open(path_to_data + 'train_labels.json') as f: 
    dict_labels = json.load(f)

with open(path_to_data + '/test_labels.json') as f: 
    test_labels = json.load(f)

In [ ]:
list_labels = list(dict_labels.values())
list_test_labels = list(test_labels.values())

In [ ]:
#loading into lists different features for the training data and making a DataFrame out of the data

trainListBrut = []
for i in range(len(dict_labels)):
  pdw = np.load(path_to_data + "train/pdw-"+str(i)+".npz")
  element = np.array([pdw['largeur'], pdw['frequence'], pdw['puissance'], pdw['phi'], pdw['theta'], pdw['date']])
  trainListBrut.append(element)



In [ ]:


testListBrut = []
for i in range(len(test_labels)):
  pdw = np.load(path_to_data + "test/pdw-"+str(i)+".npz")
  element = np.array([pdw['largeur'], pdw['frequence'], pdw['puissance'], pdw['phi'], pdw['theta'], pdw['date']])
  testListBrut.append(element)




In [ ]:
from tqdm import tqdm
#loading into lists different features for the training data and making a DataFrame out of the data

trainListBrutNew = []
for i in tqdm(range(len(dict_labels))):
  pdw = np.load(path_to_data + "train/pdw-"+str(i)+".npz")
  element = np.array([pdw['largeur'], pdw['frequence'], pdw['puissance'], np.concatenate(([0], np.diff(pdw['date'])))])
  trainListBrutNew.append(element)





100%|██████████| 2000/2000 [08:36<00:00,  3.87it/s]


In [ ]:

testListBrutNew = []
for i in tqdm(range(len(test_labels))):
  pdw = np.load(path_to_data + "test/pdw-"+str(i)+".npz")
  element = np.array([pdw['largeur'], pdw['frequence'], pdw['puissance'], np.concatenate(([0], np.diff(pdw['date'])))])
  testListBrutNew.append(element)


100%|██████████| 800/800 [10:39<00:00,  1.25it/s]


In [ ]:
with open('drive/My Drive/Projet_II/trainListBrutNew.pickle', 'wb') as handle:
    pickle.dump(trainListBrutNew, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('drive/My Drive/Projet_II/trainListBrutNew.pickle', 'rb') as handle:
    trainListBrutNew = pickle.load(handle)

In [ ]:
with open('drive/My Drive/Projet_II/testListBrutNew.pickle', 'wb') as handle:
    pickle.dump(testListBrutNew, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('drive/My Drive/Projet_II/testListBrutNew.pickle', 'rb') as handle:
    testListBrutNew = pickle.load(handle)

In [ ]:
with open('drive/My Drive/Projet_II/trainListBrut.pickle', 'wb') as handle:
    pickle.dump(trainListBrut, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('drive/My Drive/Projet_II/trainListBrut.pickle', 'rb') as handle:
    trainListBrut = pickle.load(handle)

In [ ]:
with open('drive/My Drive/Projet_II/testListBrut.pickle', 'wb') as handle:
    pickle.dump(testListBrut, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('drive/My Drive/Projet_II/testListBrut.pickle', 'rb') as handle:
    testListBrut = pickle.load(handle)

In [ ]:
#loading into lists different features for the training data and making a DataFrame out of the data

avg_largeurs=[]
avg_frequences=[]
avg_puissances=[]

var_frequences=[]
var_puissances=[]

kurtosisFreq = []

for i in range(len(dict_labels)):
  pdw = np.load("/content/drive/MyDrive/Projet_II/train/pdw-"+str(i)+".npz")
  avg_largeurs.append(np.mean(pdw['largeur']))
  avg_frequences.append(np.mean(pdw['frequence']))
  avg_puissances.append(np.mean(pdw['puissance']))
  var_frequences.append(np.var(pdw['frequence']))
  var_puissances.append(np.var(pdw['puissance']))
  kurtosisFreq.append(kurtosis(pdw['frequence']))




In [ ]:

train_completenp = np.column_stack([avg_largeurs, avg_frequences, avg_puissances, var_frequences, var_puissances, kurtosisFreq, list_labels ])
train_completeDF = pd.DataFrame(train_completenp, columns=['avg_largeurs', 'avg_frequence', 'avg_puissance', 'var_frequence', 'var_puissance', "kurtosisFreq", 'label'])

train_completeDF.label = train_completeDF.label.replace('nonmenace', 0)
train_completeDF.label = train_completeDF.label.replace('menace', 1)

In [ ]:
#loading into lists different features for the testing data and making a DataFrame out of the data
Tavg_largeurs=[]
Tavg_frequences=[]
Tavg_puissances=[]

Tvar_frequences=[]
Tvar_puissances=[]

TkurtosisFreq = []

for i in range(len(test_labels)):
  pdw = np.load("/content/drive/MyDrive/Projet_II/test/pdw-"+str(i)+".npz")
  Tavg_largeurs.append(np.mean(pdw['largeur']))
  Tavg_frequences.append(np.mean(pdw['frequence']))
  Tavg_puissances.append(np.mean(pdw['puissance']))
  Tvar_frequences.append(np.var(pdw['frequence']))
  Tvar_puissances.append(np.var(pdw['puissance']))
  TkurtosisFreq.append(kurtosis(pdw['frequence']))

In [ ]:
test_np = np.column_stack([Tavg_largeurs, Tavg_frequences, Tavg_puissances, Tvar_frequences, Tvar_puissances, TkurtosisFreq, list_test_labels])
df_test = pd.DataFrame(test_np, columns=['avg_largeurs', 'avg_frequence', 'avg_puissance', 'var_frequence', "var_puissances", 'kurtosisFreq', 'label'])

df_test.label = df_test.label.replace('nonmenace', 0)
df_test.label = df_test.label.replace('menace', 1)

In [ ]:
train_completeDF
#df_test

,avg_largeurs,avg_frequence,avg_puissance,var_frequence,var_puissance,kurtosisFreq,label
0,0.2920029761904762,2.2021488468344814,-151.16828615996047,7.936452660159338e-05,203.66206708041813,0.49458964678245243,0
1,0.13701665124884368,2.9960371178229366,-152.73364850729854,0.00015197383913408583,135.8561313424652,-0.07343399665103956,0
2,0.20196220472440946,0.9864345596615276,-171.2531592449942,1.6023495255175787e-05,147.9766704169512,0.003984194219526671,1
3,0.06901390820584145,1.6655921810586563,-149.8297143014817,4.368668344947277e-05,91.48433896645852,0.0988434486726617,0
4,0.3240181518151815,2.2050390746792115,-177.21414360174737,7.275157398452599e-05,124.60873001680986,-0.33468901192464795,0
...,...,...,...,...,...,...,...
1995,0.14399096657633245,1.247535398042877,-184.61802830883272,0.005022417419248423,174.49384773885774,-1.2253522795829597,1
1996,0.1599614624505929,1.8785500187550042,-147.78948486198558,0.011934391119257374,122.30999316191645,-1.2036825319228355,1
1997,0.028000492004920056,1.9177639254479806,-148.94320882344323,5.8086301056794836e-05,173.90912339106595,-0.13292277449769685,1
1998,0.10700590667454227,2.040005034533262,-151.17010967029202,6.482565744160364e-05,115.90361023820188,-0.04473800828436403,1


In [ ]:
train_completeDF.to_csv('df_Train.csv')
!cp df_Train.csv "drive/My Drive/Projet_II/"

df_test.to_csv('df_Test.csv')
!cp df_Test.csv "drive/My Drive/Projet_II/"

In [ ]:
train_completeDF = pd.read_csv('/content/drive/MyDrive/Projet_II/df_Train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Projet_II/df_Test.csv')

In [ ]:
x = train_completeDF.drop(['label'], axis=1).to_numpy()
y = np.array(train_completeDF.label)

In [ ]:
xtest = df_test.drop(['label'], axis=1).to_numpy()
ytest = np.array(df_test.label)

#Testing ResNet on brute data

#Prep data for ResNet Brut

In [ ]:
import torch
import torch.nn as nn

In [ ]:
list_labels = list(dict_labels.values())
list_test_labels = list(test_labels.values())

In [ ]:
lastTrain = [torch.from_numpy(item).float() for item in trainListBrut]


In [ ]:
lastTest = [torch.from_numpy(item).float() for item in testListBrut]

transforming the labels into integers

In [ ]:

list_labels = [i.replace('nonmenace', "0") for i in list_labels]
list_labels = [i.replace('menace', "1") for i in list_labels]
list_labels  = list(map(int, list_labels ))

In [ ]:
list_labelsTest = [i.replace('nonmenace', "0") for i in list_test_labels]
list_labelsTest = [i.replace('menace', "1") for i in list_labelsTest]
list_labelsTest  = list(map(int, list_labelsTest ))

In [ ]:
list_labels_train_tensor = torch.tensor(list_labels)

In [ ]:
list_labels_test_tensor = torch.tensor(list_labelsTest)

#Padding the signals with 0

In [ ]:
shapes = [np.shape(x) for x in lastTrain]
max_shape = tuple(np.amax(shapes, axis=0))


In [ ]:
max_shape

(6, 20014)

In [ ]:
shapes = [np.shape(x) for x in lastTest]
max_shapeTest = tuple(np.amax(shapes, axis=0))
max_shapeTest

(6, 19065)

In [ ]:
# desired size
size = (6, 20014)

# padding value
pad_value = 0

# list to store padded arrays
padded_list_0 = []

for array in lastTrain:
    # calculate amount of padding needed
    pad_width = [(0, size[0]-array.shape[0]), (0, size[1]-array.shape[1])]
    # use numpy.pad() to add padding
    padded_array = np.pad(array, pad_width, mode='constant', constant_values=pad_value)
    padded_list_0.append(padded_array)

In [ ]:
padded_list_0[0]

array([[   0.291     ,    0.291     ,    0.293     , ...,    0.        ,
           0.        ,    0.        ],
       [   2.2220113 ,    2.1887379 ,    2.2040505 , ...,    0.        ,
           0.        ,    0.        ],
       [-125.58249   , -133.03972   , -149.99168   , ...,    0.        ,
           0.        ,    0.        ],
       [   1.6789358 ,    1.8065993 ,    1.380749  , ...,    0.        ,
           0.        ,    0.        ],
       [   0.957886  ,    0.77572364,    0.8238563 , ...,    0.        ,
           0.        ,    0.        ],
       [   1.31      ,   24.985     ,   36.823     , ...,    0.        ,
           0.        ,    0.        ]], dtype=float32)

In [ ]:
padded_listTest_0 = []

for array in lastTest:
    # calculate amount of padding needed
    pad_width = [(0, size[0]-array.shape[0]), (0, size[1]-array.shape[1])]
    # use numpy.pad() to add padding
    padded_array = np.pad(array, pad_width, mode='constant', constant_values=pad_value)
    padded_listTest_0.append(padded_array)

In [ ]:
lastTrain_pad_0 = torch.tensor(padded_list_0)

<ipython-input-130-7ffda439b419>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  lastTrain_pad_0 = torch.tensor(padded_list_0)


In [ ]:
lastTest_pad_0 = torch.tensor(padded_listTest_0)


In [ ]:
batch_size = 128

#Padding with -1

In [ ]:
# desired size
size = (6, 20014)

# padding value
pad_value = -1

# list to store padded arrays
padded_list_1 = []

for array in lastTrain:
    # calculate amount of padding needed
    pad_width = [(0, size[0]-array.shape[0]), (0, size[1]-array.shape[1])]
    # use numpy.pad() to add padding
    padded_array = np.pad(array, pad_width, mode='constant', constant_values=pad_value)
    padded_list_1.append(padded_array)

In [ ]:
padded_listTest_1 = []

for array in lastTest:
    # calculate amount of padding needed
    pad_width = [(0, size[0]-array.shape[0]), (0, size[1]-array.shape[1])]
    # use numpy.pad() to add padding
    padded_array = np.pad(array, pad_width, mode='constant', constant_values=pad_value)
    padded_listTest_1.append(padded_array)

In [ ]:
lastTest_pad_1 = torch.tensor(padded_listTest_1)
lastTrain_pad_1 = torch.tensor(padded_list_1)

#data loaders for paddings

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

my_datasetTrain_0 = TensorDataset(lastTrain_pad_0,list_labels_train_tensor ) # create your datset
my_dataloaderTrain_0 = DataLoader(my_datasetTrain_0, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

In [ ]:

my_datasetTest_0 = TensorDataset(lastTest_pad_0,list_labels_test_tensor) # create your datset
my_dataloaderTest_0 = DataLoader(my_datasetTest_0, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

my_datasetTrain_1 = TensorDataset(lastTrain_pad_1,list_labels_train_tensor ) # create your datset
my_dataloaderTrain_1 = DataLoader(my_datasetTrain_1, batch_size=batch_size,
                                          shuffle=True)

my_datasetTest_1 = TensorDataset(lastTest_pad_1,list_labels_test_tensor) # create your datset
my_dataloaderTest_1 = DataLoader(my_datasetTest_1, batch_size=batch_size,
                                          shuffle=True)


# Pooling 

In [ ]:


# Create a 1D adaptive max pooling layer
adaptive_max_pool = torch.nn.AdaptiveMaxPool1d(output_size=2000)

# Apply the pooling layer to the list of signals
pooled_signals_Train = [adaptive_max_pool(sig.unsqueeze(0)) for sig in lastTrain]
pooled_signals_Test = [adaptive_max_pool(sig.unsqueeze(0)) for sig in lastTest]

# The output will be a list of tensors of size (batch_size, num_channels, output_size)

In [ ]:
max_length = max(sig.shape[1] for sig in lastTrain)
print(max_length)

20014


In [ ]:
sum = 0
for i in lastTrain:
  sum += i.shape[1]
print(sum/len(lastTrain))

1698.798


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

signals_train_tensor = torch.stack(pooled_signals_Train)


my_datasetTrain_pool= TensorDataset(signals_train_tensor, list_labels_train_tensor) # create your datset
my_dataloaderTrain_pool = DataLoader(my_datasetTrain_pool, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

signals_test_tensor = torch.stack(pooled_signals_Test)

my_datasetTest_pool = TensorDataset(signals_test_tensor,list_labels_test_tensor) # create your datset
my_dataloaderTest_pool = DataLoader(my_datasetTest_pool, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

#The ResNet model 

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm1d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv1d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm1d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 1):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv1d(6, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm1d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool1d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        #self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        #self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        #self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.avgpool = nn.AvgPool1d(7, stride=1)
        #self.fc = nn.Linear(in_features=319872, out_features=1, bias=False)
        #self.fc = nn.Linear(in_features=26816, out_features=1, bias=False)
        self.fc = nn.Linear(in_features=31616, out_features=1, bias=False)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm1d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        #x = self.layer1(x)
        # x = self.layer2(x)
        # x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        #x = x.view(-1,x.shape[1]*x.shape[2]) 
        #print(x.shape)
        x = self.fc(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
model = ResNet(ResidualBlock, [5]).to(device)


In [ ]:
sum([element.numel() for element in model.parameters()])

159296

In [ ]:
model

ResNet(
  (conv1): Sequential(
    (0): Conv1d(6, 64, kernel_size=(7,), stride=(2,), padding=(3,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer0): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [ ]:
num_epochs = 50
learning_rate= 0.0001

55.5% accuracy with padding and resNet with 5 residual blocks 20 epochs


65% padding avec adam

---




76.75% accuracy avec pooling dim 1698, 90 epochs lr 0.0001, 5 residual blocks \\
76% for a resnet 15 residual blocks, 150 epochs 0.0001 lr

78.125% avec Adam opitimzer

In [ ]:
yTrue = []
yPred = []

from torch.optim.lr_scheduler import StepLR
# Loss and optimizer
lossFun = torch.nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1, verbose=True) 

# Train the model
total_step = len(my_dataloaderTrain_0)

acc = []
for epoch in range(num_epochs):
    for i, (sigs, labels) in enumerate(my_dataloaderTrain_0):  
        # Move tensors to the configured device
        sigs = sigs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(sigs.squeeze(1))
        # Convert output from sigmoid to binary classes
        predicted = (sigmoid(outputs) > 0.5).float()
        loss = lossFun(outputs.squeeze(1).float(), labels.float())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #scheduler.step()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation

    with torch.no_grad():
        correct = 0
        total = 0
        for sigs, labels in (my_dataloaderTest_0):
            sigs = sigs.to(device)
            labels = labels.to(device)
            outputs = model(sigs.squeeze(1))
            predicted = (sigmoid(outputs) > 0.5).int()
            total += labels.size(0)
            correct += (predicted.squeeze(1) == labels).sum().item()
            yTrue.append(labels.int().cpu())
            yPred.append(predicted.squeeze(1).cpu())
        acc.append(100 * correct / total)
        print('Accuracy of the network: {} %'.format(100 * correct / total)) 

        print('\n Recall', recall_score(np.array(torch.stack(yTrue, dim=0).cpu()),np.array(torch.stack(yPred, dim=0).cpu()),average='micro'))
        print('\n Precision', precision_score(np.array(torch.stack(yTrue).squeeze(1).cpu()),np.array(torch.stack(yPred).squeeze(1).cpu()),average='micro'))

print("maximum Accuracy : ", max(acc))


Adjusting learning rate of group 0 to 1.0000e-03.
Epoch [1/50], Loss: 6.4511
Accuracy of the network: 50.520833333333336 %

 Recall 0.9216216216216216

 Precision 0.49277456647398843
Epoch [2/50], Loss: 4.3401
Accuracy of the network: 54.557291666666664 %

 Recall 0.6576819407008087

 Precision 0.5067497403946002
Epoch [3/50], Loss: 1.7591
Accuracy of the network: 52.864583333333336 %

 Recall 0.5899280575539568

 Precision 0.5081332300542215
Epoch [4/50], Loss: 2.4654
Accuracy of the network: 51.432291666666664 %

 Recall 0.680349932705249

 Precision 0.5055
Epoch [5/50], Loss: 1.9340
Accuracy of the network: 53.776041666666664 %

 Recall 0.5748922413793104

 Precision 0.508822126847878
Epoch [6/50], Loss: 1.8149
Accuracy of the network: 48.828125 %

 Recall 0.6286484059272563

 Precision 0.5026929982046678
Epoch [7/50], Loss: 1.8718
Accuracy of the network: 52.473958333333336 %

 Recall 0.5714285714285714

 Precision 0.5033921302578019
Epoch [8/50], Loss: 0.8953
Accuracy of the netwo

#Testing processed signals (Stat values) with a ResNet

In [ ]:
x = torch.from_numpy(x)
y = torch.from_numpy(y)

xtest = torch.from_numpy(xtest)
ytest = torch.from_numpy(ytest)

In [ ]:
batch_size = 36

In [ ]:
my_datasetTrain = TensorDataset(x,y) # create your datset
my_dataloaderTrain = DataLoader(my_datasetTrain, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

In [ ]:
my_datasetTest = TensorDataset(xtest,ytest) # create your datset
my_dataloaderTest = DataLoader(my_datasetTest, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm1d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv1d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm1d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 1):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv1d(1, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm1d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool1d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        #self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        #self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        #self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.avgpool = nn.AvgPool1d(1, stride=1)
        #self.fc = nn.Linear(in_features=319872, out_features=1, bias=False)
        #self.fc = nn.Linear(in_features=26816, out_features=1, bias=False)
        #self.fc = nn.Linear(in_features=31616, out_features=1, bias=False)
        self.fc = nn.Linear(in_features=128, out_features=1, bias=False)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm1d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        #x = self.layer1(x)
        # x = self.layer2(x)
        # x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        #x = x.view(-1,x.shape[1]*x.shape[2]) 
        #print(x.shape)
        x = self.fc(x)

        return x






In [ ]:
model = ResNet(ResidualBlock, [5]).to(device)

In [ ]:
list1 = [element.numel() for element in model.parameters()]
summ = 0
for i in list1:
  summ += i
summ

125568

In [ ]:
num_epochs = 300
learning_rate= 0.01

66.875% with 150 epochs, 5 resblocks \\
79.375% 300 epochs 

83.125% 300 epochs Adam optimizer

In [ ]:
from sklearn.metrics import recall_score, precision_score
yTrue = []
yPred =[]
# Loss and optimizer
lossFun = torch.nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9) 

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay = 0.0001)

scheduler = StepLR(optimizer, step_size=100, gamma=0.1, verbose=True) 

# Train the model
total_step = len(my_dataloaderTrain)

acc = []
for epoch in range(num_epochs):
    for i, (sigs, labels) in enumerate(my_dataloaderTrain):  
        # Move tensors to the configured device
        sigs = sigs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(sigs.unsqueeze(1).float())
        # Convert output from sigmoid to binary classes
        predicted = (sigmoid(outputs) > 0.5).float()
        loss = lossFun(outputs.squeeze(1).float(), labels.float())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation

    with torch.no_grad():
        correct = 0
        total = 0
        for sigs, labels in (my_dataloaderTest):
            sigs = sigs.to(device)
            labels = labels.to(device)
            outputs = model(sigs.unsqueeze(1).float())
            predicted = (sigmoid(outputs) > 0.5).int()
            total += labels.size(0)
            correct += (predicted.squeeze(1) == labels).sum().item()
            yTrue.append(labels.int().cpu())
            #print("labels", labels.shape)
            #print("predicted", predicted.squeeze(1).shape)
            yPred.append(predicted.squeeze(1).cpu())
        acc.append(100 * correct / total)
        print('Accuracy of the network: {} %'.format(100 * correct / total))
        #print(np.array(torch.stack(yTrue, dim=0).cpu()))

        print('\n Recall', recall_score(np.array(torch.stack(yTrue, dim=0).cpu()),np.array(torch.stack(yPred, dim=0).cpu()),average='micro'))
        print('\n Precision', precision_score(np.array(torch.stack(yTrue).squeeze(1).cpu()),np.array(torch.stack(yPred).squeeze(1).cpu()),average='micro'))

print("maximum Accuracy : ", max(acc))


Adjusting learning rate of group 0 to 1.0000e-02.
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [1/300], Loss: 0.6764
Accuracy of the network: 52.27272727272727 %

 Recall 0.5380577427821522

 Precision 0.5036855036855037
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [2/300], Loss: 0.7508
Accuracy of the network: 50.63131313131313 %

 Recall 0.6259842519685039

 Precision 0.4963579604578564
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [3/300], Loss: 0.6871
Accuracy of the network: 58.71212121212121 %

 Recall 0.4645048203330412

 Precision 0.5221674876847291
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [4/300], Loss: 0.6549
Accuracy of the network: 57.323232323232325 %

 Recall 0.4174884944115713

 Precision 0.536770921386306
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [5/300], Loss: 0.6212
Accuracy of the network: 61.48989898989899 %

 Recall 0.3742105263157895

 Precision 0.5638382236320381
Adjusting learning rate of group 0 to 1.

#Testing fully connected layers for the preprocessed data

In [ ]:
x = torch.from_numpy(x)
y = torch.from_numpy(y)

xtest = torch.from_numpy(xtest)
ytest = torch.from_numpy(ytest)

In [ ]:
batch_size = 36

In [ ]:
my_datasetTrain = TensorDataset(x,y) # create your datset
my_dataloaderTrain = DataLoader(my_datasetTrain, batch_size=batch_size,
                                          shuffle=True)

In [ ]:
my_datasetTest = TensorDataset(xtest,ytest) # create your datset
my_dataloaderTest = DataLoader(my_datasetTest, batch_size=batch_size,
                                          shuffle=True)

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(7, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.sigmoid(x)



In [ ]:
# instantiate the model
model = BinaryClassifier()

# define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay = 0.0001)

In [ ]:
num_epochs = 600
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
model

BinaryClassifier(
  (fc1): Linear(in_features=7, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
sum([element.numel() for element in model.parameters()])

801

acc : 78.125% over 500 epochs, no regularization

78.75% over 500 epochs with regularization 0.0001

80.375% over 600 epochs

In [ ]:
scheduler = StepLR(optimizer, step_size=200, gamma=0.1, verbose=True) 

acc = []
for epoch in range(num_epochs):
    for i, (sigs, labels) in enumerate(my_dataloaderTrain):  
        # Move tensors to the configured device
        sigs = sigs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(sigs.unsqueeze(1).float())
        # Convert output from sigmoid to binary classes
        predicted = (sigmoid(outputs) > 0.5).float()
        #print(outputs.squeeze(1).squeeze(1).shape)
        loss = criterion(outputs.squeeze(1).squeeze(1).float(), labels.float())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation

    with torch.no_grad():
        correct = 0
        total = 0
        for sigs, labels in (my_dataloaderTest):
            sigs = sigs.to(device)
            labels = labels.to(device)
            outputs = model(sigs.float())
            # print(outputs)
            # break
            predicted = (outputs > 0.5).int()
            total += labels.size(0)
            correct += (predicted.squeeze(1) == labels).sum().item()
        acc.append(100 * correct / total)
        print('Accuracy of the network: {} %'.format(100 * correct / total)) 

print("maximum Accuracy : ", max(acc))

Adjusting learning rate of group 0 to 1.0000e-02.
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [1/600], Loss: 0.6928
Accuracy of the network: 49.5 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [2/600], Loss: 0.6779
Accuracy of the network: 54.375 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [3/600], Loss: 0.6023
Accuracy of the network: 57.75 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [4/600], Loss: 0.6888
Accuracy of the network: 51.125 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [5/600], Loss: 0.7279
Accuracy of the network: 60.0 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [6/600], Loss: 0.6953
Accuracy of the network: 60.25 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [7/600], Loss: 0.6856
Accuracy of the network: 66.625 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [8/600], Loss: 0.5658
Accuracy of the network: 71.375 %
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch [9/6

#Testing only with date difference, frequency, power, width 

In [ ]:
mean_tensor = torch.tensor([0.0871414782687525, 1.9893857392132595, -154.90684374070747, 5.875007060286158], dtype=torch.float32)
std_tensor = torch.tensor([0.0845755502639984, 0.8923242781137891, 19.407849095325062, 8.705384067948797], dtype=torch.float32)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
list_labels = list(dict_labels.values())
list_test_labels = list(test_labels.values())

In [ ]:
lastTrain = [(torch.from_numpy(item).unsqueeze(0)- mean_tensor.unsqueeze(1)) / std_tensor.unsqueeze(1)  for item in trainListBrutNew]


In [ ]:

lastTrain = [torch.from_numpy(item)   for item in trainListBrutNew]


In [ ]:
lastTest = [torch.from_numpy(item)  for item in testListBrutNew]


In [ ]:


# Create a 1D adaptive max pooling layer
adaptive_max_pool = torch.nn.AdaptiveMaxPool1d(output_size=1600)

# Apply the pooling layer to the list of signals
pooled_signals_Train = [adaptive_max_pool(sig) for sig in lastTrain]
pooled_signals_Test = [adaptive_max_pool(sig) for sig in lastTest]

# The output will be a list of tensors of size (batch_size, num_channels, output_size)

In [ ]:

list_labels = [i.replace('nonmenace', "0") for i in list_labels]
list_labels = [i.replace('menace', "1") for i in list_labels]
list_labels  = list(map(int, list_labels ))

In [ ]:
list_labelsTest = [i.replace('nonmenace', "0") for i in list_test_labels]
list_labelsTest = [i.replace('menace', "1") for i in list_labelsTest]
list_labelsTest  = list(map(int, list_labelsTest ))

In [ ]:
list_labels_train_tensor = torch.tensor(list_labels)

In [ ]:
list_labels_test_tensor = torch.tensor(list_labelsTest)

In [ ]:
batch_size = 126

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

signals_train_tensor = torch.stack(pooled_signals_Train)


my_datasetTrain_pool= TensorDataset(signals_train_tensor, list_labels_train_tensor) # create your datset
my_dataloaderTrain_pool = DataLoader(my_datasetTrain_pool, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

signals_test_tensor = torch.stack(pooled_signals_Test)

my_datasetTest_pool = TensorDataset(signals_test_tensor,list_labels_test_tensor) # create your datset
my_dataloaderTest_pool = DataLoader(my_datasetTest_pool, batch_size=batch_size,
                                          shuffle=True, drop_last = True)

number parameters : 152000

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm1d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv1d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm1d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 1):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv1d(4, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm1d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool1d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        #self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        #self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        #self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.avgpool = nn.AvgPool1d(7, stride=1)
        #self.fc = nn.Linear(in_features=319872, out_features=1, bias=False)
        #self.fc = nn.Linear(in_features=26816, out_features=1, bias=False)
        self.fc = nn.Linear(in_features=25216, out_features=1, bias=False)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm1d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        #x = self.layer1(x)
        # x = self.layer2(x)
        # x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        #x = x.view(-1,x.shape[1]*x.shape[2]) 
        #print(x.shape)
        x = self.fc(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:

model = ResNet(ResidualBlock, [5]).to(device)

In [ ]:
sum([element.numel() for element in model.parameters()])

152000

In [ ]:
num_epochs = 50


80% sur 50 epochs
80.25% sans normaliser les donnees

In [ ]:
yTrue = []
yPred = []
from sklearn.metrics import recall_score, precision_score
from torch.optim.lr_scheduler import StepLR
# Loss and optimizer
lossFun = torch.nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#scheduler = StepLR(optimizer, step_size=50, gamma=0.1, verbose=True) 

# Train the model
total_step = len(my_dataloaderTrain_pool)

acc = []
for epoch in range(num_epochs):
    for i, (sigs, labels) in enumerate(my_dataloaderTrain_pool):  
        # Move tensors to the configured device
        sigs = sigs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(sigs.squeeze(1).float())
        # Convert output from sigmoid to binary classes
        predicted = (sigmoid(outputs) > 0.5).float()
        loss = lossFun(outputs.squeeze(1).float(), labels.float())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #scheduler.step()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation

    with torch.no_grad():
        correct = 0
        total = 0
        for sigs, labels in (my_dataloaderTest_pool):
            sigs = sigs.to(device)
            labels = labels.to(device)
            outputs = model(sigs.squeeze(1).float())
            predicted = (sigmoid(outputs) > 0.5).int()
            total += labels.size(0)
            correct += (predicted.squeeze(1) == labels).sum().item()
            yTrue.append(labels.int().cpu())
            yPred.append(predicted.squeeze(1).cpu())
        acc.append(100 * correct / total)
        print('Accuracy of the network: {} %'.format(100 * correct / total)) 
        print('\n Recall', recall_score(np.array(torch.stack(yTrue, dim=0).cpu()),np.array(torch.stack(yPred, dim=0).cpu()),average='micro'))
        print('\n Precision', precision_score(np.array(torch.stack(yTrue).squeeze(1).cpu()),np.array(torch.stack(yPred).squeeze(1).cpu()),average='micro'))

print("maximum Accuracy : ", max(acc))


Epoch [1/50], Loss: 1.2482
Accuracy of the network: 73.41269841269842 %

 Recall 0.5883977900552486

 Precision 0.8037735849056604
Epoch [2/50], Loss: 0.3681
Accuracy of the network: 75.26455026455027 %

 Recall 0.7075862068965517

 Precision 0.7445573294629898
Epoch [3/50], Loss: 0.3788
Accuracy of the network: 78.43915343915344 %

 Recall 0.7170506912442396

 Precision 0.761252446183953
Epoch [4/50], Loss: 0.4482
Accuracy of the network: 79.23280423280423 %

 Recall 0.7570738440303658

 Precision 0.7549896765313145
Epoch [5/50], Loss: 0.3085
Accuracy of the network: 75.92592592592592 %

 Recall 0.7920353982300885

 Precision 0.7358684480986639
Epoch [6/50], Loss: 0.3331
Accuracy of the network: 79.8941798941799 %

 Recall 0.7981609195402298

 Precision 0.7421975203078238
Epoch [7/50], Loss: 0.2851
Accuracy of the network: 77.38095238095238 %

 Recall 0.7794001578531965

 Precision 0.7512362114872575
Epoch [8/50], Loss: 0.2970
Accuracy of the network: 78.70370370370371 %

 Recall 0.79